In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

import torch
from safetensors.torch import load_file
import pandas as pd
import os
from tqdm import tqdm 

versi_model = 12

df = pd.read_csv("dataset/barokah.csv", sep="|")

In [11]:
# Encode labels
df['label'] = df['answer'].astype('category').cat.codes
label_dict = dict(enumerate(df['answer'].astype('category').cat.categories))


tokenizer = BertTokenizer.from_pretrained("./models/model v"+str(versi_model))
model = BertForSequenceClassification.from_pretrained("./models/model v"+str(versi_model))


def get_answer(question):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model(**inputs)
    answer = torch.argmax(outputs.logits).item()
    return label_dict[answer]
    # return outputs

In [8]:
df.head()

question                                             answer  \
0      pembayaran online  Pembayaran online saat ini dapat dilakukan mel...   
1      pengiriman barang  Pengiriman barang kini dapat dilacak secara re...   
2  sistem keamanan rumah  Sistem keamanan rumah modern menggunakan tekno...   
3             e-learning  E-learning memungkinkan siswa untuk mengakses ...   
4   perpustakaan digital  Perpustakaan digital menyediakan akses ke ribu...   

   label  
0    629  
1    638  
2    768  
3    254  
4    664

In [12]:
# List untuk menampung hasil prediksi
predicted_labels = []

# Iterasi pada setiap baris data
for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row['question']
    predicted_label = get_answer(text)
    predicted_labels.append(predicted_label)

# Menambahkan hasil prediksi ke dataframe baru
df['Predicted_Label'] = predicted_labels

100%|██████████| 5000/5000 [02:57<00:00, 28.23it/s]


In [13]:
df.head()

question                                             answer  \
0      pembayaran online  Pembayaran online saat ini dapat dilakukan mel...   
1      pengiriman barang  Pengiriman barang kini dapat dilacak secara re...   
2  sistem keamanan rumah  Sistem keamanan rumah modern menggunakan tekno...   
3             e-learning  E-learning memungkinkan siswa untuk mengakses ...   
4   perpustakaan digital  Perpustakaan digital menyediakan akses ke ribu...   

   label                                    Predicted_Label  
0    629  Pembayaran online saat ini dapat dilakukan mel...  
1    638  Pengiriman barang kini dapat dilacak secara re...  
2    768  Sistem keamanan rumah modern menggunakan tekno...  
3    254  E-learning memungkinkan siswa untuk mengakses ...  
4    664  Perpustakaan digital menyediakan akses ke ribu...

In [15]:
# Hitung akurasi
correct_predictions = (df['Predicted_Label'] == df['answer']).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions * 100
print(f'Correct predictions: {correct_predictions}/{total_predictions}')
print(f'Accuracy: {accuracy:.2f}%')

Correct predictions: 4625/5000
Accuracy: 92.50%
